In [10]:
from cnn_model import *
import sys
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

def cross_entropy(y_predict, label):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_predict, labels=label))
    #優化器: 使cross entropy最小化
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    #預測準確度
    #argmax: 得到Tensor中最大值的index
    correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(label,1))
    #神經網路準確率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return train_step, accuracy




model = mnist_cnn_model()
y = model.run_cnn()


train_step, accuracy = cross_entropy(y, model.label_placeholder)
train_accuracy = 0.0



#initialize
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

#train for 1000 times
for i in range(200):
    batch = mnist.train.next_batch(100)
    #跑訓練集，利用optimizer將結果導到較好的地方
    train_step.run(session=sess, feed_dict={model.img_placeholder: batch[0],
                                            model.label_placeholder: batch[1], model.keep_prob: 0.7})
    
    if i%10 == 0:
        #輸出準確度
        train_accuracy = accuracy.eval(session=sess, feed_dict={model.img_placeholder:batch[0], 
                                        model.label_placeholder: batch[1], model.keep_prob: 1.0})
        
        print("step {}, training accuracy {:}".format(i, train_accuracy))

print("Accuracy: ", sess.run(accuracy, feed_dict={model.img_placeholder: mnist.test.images,
                                model.label_placeholder: mnist.test.labels, model.keep_prob: 1.0}))





print("test:", sess.run(tf.argmax(mnist.test.labels,1)))
print("model:", sess.run(tf.argmax(y, 1), feed_dict={model.img_placeholder: mnist.test.images,
                                            model.label_placeholder: mnist.test.labels, 
                                            model.keep_prob: 1.0}))


# Saving
'''
import pickle
export_dir =  "my_net/save_net.pickle"
model.details()
with open(export_dir, "wb") as f:
    pickle.dump(model, f)
    print("Model saved in path: ", export_dir)


'''
export_dir =  "my_net/save_net.ckpt"
# saver = tf.train.Saver()
saver = tf.train.Saver({
            "W_conv1": model.W_conv1, 
            "W_conv2": model.W_conv2, 
            "b_conv1": model.b_conv1, 
            "b_conv2": model.b_conv2, 
            "W_fc1": model.W_fc1, 
            "W_fc2": model.W_fc2, 
            "b_fc1": model.b_fc1, 
            "b_fc2": model.b_fc2
        })
sess.run(init)
save_path = saver.save(sess, export_dir)
#save_path = saver.save(sess, export_dir, global_step = 200)
print("Model saved in path: ", save_path)


def details(model, sess):
    print("{:30}:{}".format("convolution weight 1", sess.run(model.W_conv1)))
    #print("{:30}:{}".format("convolution bias 1", sess.run(model.b_conv1)))
    print("{:30}:{}".format("convolution weight 2", sess.run(model.W_conv2)))
    #print("{:30}:{}".format("convolution bias 2", sess.run(model.b_conv2)))

    print("{:30}:{}".format("full connected weight 1", sess.run(model.W_fc1)))
    #print("{:30}:{}".format("full connected bias 1", sess.run(model.b_fc1)))
    print("{:30}:{}".format("full connected weight 2", sess.run(model.W_fc2)))
    #print("{:30}:{}".format("full connected bias 2", sess.run(model.b_fc2)))
    return

# print(sess.run(y , {model.img_placeholder: mnist.test.images,
#                                             model.label_placeholder: mnist.test.labels, 
#                                             model.keep_prob: 1.0}))

details(model, sess)


step 0, training accuracy 0.07000000029802322
step 10, training accuracy 0.15000000596046448
step 20, training accuracy 0.3499999940395355
step 30, training accuracy 0.33000001311302185
step 40, training accuracy 0.3700000047683716
step 50, training accuracy 0.36000001430511475
step 60, training accuracy 0.38999998569488525
step 70, training accuracy 0.5099999904632568
step 80, training accuracy 0.38999998569488525
step 90, training accuracy 0.49000000953674316
step 100, training accuracy 0.6000000238418579
step 110, training accuracy 0.47999998927116394
step 120, training accuracy 0.7400000095367432
step 130, training accuracy 0.5699999928474426
step 140, training accuracy 0.6100000143051147
step 150, training accuracy 0.6499999761581421
step 160, training accuracy 0.5299999713897705
step 170, training accuracy 0.6800000071525574
step 180, training accuracy 0.7300000190734863
step 190, training accuracy 0.6000000238418579
Accuracy:  0.7267
test: [7 2 1 ... 4 5 6]
model: [7 2 2 ... 4 5

convolution weight 2          :[[[[ 1.43835172e-02 -1.22847473e-02  6.87649474e-02 ...  4.07178923e-02
     5.72057739e-02  2.28642183e-03]
   [-1.49812549e-01  2.92232819e-02 -5.08959368e-02 ... -3.93813066e-02
     6.21011145e-02 -1.37392983e-01]
   [-8.43494833e-02  5.49789965e-02 -5.23699708e-02 ... -3.88903096e-02
    -4.64985818e-02  4.35863100e-02]
   ...
   [-2.82119401e-02  3.18050347e-02  9.96211171e-02 ... -2.94300797e-03
     9.65226591e-02 -1.18760867e-02]
   [-3.77663746e-02 -7.58270845e-02 -1.15080677e-01 ... -9.59256664e-02
    -7.95164853e-02  3.23284604e-02]
   [-1.31781161e-01 -7.68465400e-02 -5.91523536e-02 ... -1.08051533e-02
     1.30405650e-02 -7.34946653e-02]]

  [[ 3.40382867e-02 -1.98973790e-02  1.00833751e-01 ... -5.56305163e-02
    -7.85947815e-02 -5.99210262e-02]
   [-1.65255547e-01  1.15671232e-01  8.82853493e-02 ... -1.70153484e-01
     1.62167083e-02  2.65187472e-02]
   [-5.24558499e-02  7.08556622e-02  1.41838389e-02 ... -1.38094738e-01
     1.06619835e